In [1]:
import pandas as pd
import yfinance as yf
import numpy as np
from scipy.stats import norm
from IPython.display import display

#pd.set_option('display.float_format', '{:,.2f}'.format)

# 1)

In [2]:
data = pd.read_excel('DatosExamen.xlsx').set_index('Date')
data

,ABT,KXI,QQQ,X,XOM
Date,,,,,
2020-01-02,79.689949,49.628918,209.976120,10.506616,55.631172
2020-01-03,78.718468,49.584068,208.052765,10.428932,55.183929
2020-01-06,79.130875,49.736576,209.393250,10.477486,55.607639
2020-01-07,78.690964,49.413612,209.364136,10.807637,55.152542
2020-01-08,79.011749,49.404640,210.937775,10.778504,54.320824
...,...,...,...,...,...
2024-12-04,115.970001,63.639999,523.260010,38.389999,114.279999
2024-12-05,115.589996,64.040001,521.809998,38.849998,114.779999
2024-12-06,115.599998,63.770000,526.479980,38.450001,113.570000


In [3]:
def VaR_CVaR_Stocks(data, n_stocks, conf, long):
     
     valor_posicion = n_stocks * data.iloc[-1]
     weights = [valor_posicion.iloc[i]/valor_posicion.sum() for i in range(len(n_stocks))]  
     rt_assets = data.pct_change().dropna()
     rt_port = rt_assets.mul(weights).sum(axis=1)

     var = np.percentile(rt_port,100-conf) if long else np.percentile(rt_port,conf)
     cvar = rt_port[rt_port < var].mean() if long else rt_port[rt_port > var].mean()
          
     var_m = var * valor_posicion.sum()
     cvar_m = cvar * valor_posicion.sum()
     
     resultados_df = pd.DataFrame({"Métrica": ["VaR", "CVaR"], "Porcentual (%)": [var * 100, cvar * 100], "Monetario ($)": [var_m, cvar_m]})

     return resultados_df

In [4]:
positions = [5432, 8972, 3243, 7456, 5931] # En mismo orden que Stocks

long = True #True o False

conf = 95 #[0, 100]

np.round(VaR_CVaR_Stocks(data, positions, conf, long)['Monetario ($)'][1], 0)

-120212.0

In [5]:
rt = data.pct_change().dropna()
rt.cov()

,ABT,KXI,QQQ,X,XOM
ABT,0.000273,0.000105,0.000146,0.000107,0.000078
KXI,0.000105,0.000101,0.000109,0.000124,0.000094
QQQ,0.000146,0.000109,0.000262,0.000208,0.000112
X,0.000107,0.000124,0.000208,0.001485,0.000371
XOM,0.000078,0.000094,0.000112,0.000371,0.000473


In [7]:
w_0

ABT    0.164306
KXI    0.149696
QQQ    0.442144
X      0.068850
XOM    0.175004
Name: 2024-12-10 00:00:00, dtype: float64

In [6]:
rt = data.pct_change().dropna()
position_value = positions * data.iloc[-1]
portfolio_value = position_value.sum()
w_0 = position_value / portfolio_value
w_t = [0.1625, 0.4406, 0.2104, 0.0075, 0.1790]

In [8]:
def rebalanceo(initial_weights, optimal_weights):
     
     df = pd.DataFrame({"Initial Weights": initial_weights.values*100, "Optimal Weights": np.array(optimal_weights)*100, "Stocks difference": np.floor((optimal_weights - initial_weights) * portfolio_value / data.iloc[-1])})

     df['Buy/Sell'] = df['Stocks difference'].apply(lambda x: 'Buy' if x > 0 else ('Sell' if x < 0 else 'NA')) if long else df['Stocks difference'].apply(lambda x: 'Sell' if x > 0 else ('Buy' if x < 0 else 'NA'))
     
     return df

In [9]:
rebalanceo(w_0, w_t)

,Initial Weights,Optimal Weights,Stocks difference,Buy/Sell
ABT,16.430637,16.25,-60.0,Sell
KXI,14.969576,44.06,17435.0,Buy
QQQ,44.214400,21.04,-1700.0,Sell
X,6.884951,0.75,-6644.0,Sell
XOM,17.500436,17.90,135.0,Buy


# 6)

In [23]:
data = pd.read_excel('DatosExamen.xlsx', sheet_name=2).set_index('Date')[['JPYUSD', 'MXNUSD']]
data

,JPYUSD,MXNUSD
Date,,
2020-01-01,0.01,0.05
2020-01-02,0.01,0.05
2020-01-03,0.01,0.05
2020-01-06,0.01,0.05
2020-01-07,0.01,0.05
...,...,...
2024-12-03,0.01,0.05
2024-12-04,0.01,0.05
2024-12-05,0.01,0.05


In [24]:
def VaR_CVaR_Fx(data, positions, conf, long):
     port = data * positions 
     total = port.sum(axis=1)
     rt = total.pct_change().dropna()

     var = np.percentile(rt,100-conf) if long else np.percentile(rt,conf)
     cvar = rt[rt < var].mean() if long else rt[rt > var].mean()

     var_m = var * total.iloc[-1]
     cvar_m = cvar * total.iloc[-1]

     resultados_df = pd.DataFrame({"Métrica": ["VaR", "CVaR"], "Porcentual (%)": [var, cvar], "Monetario ($)": [var_m, cvar_m]})

     return resultados_df

In [34]:
positions = [73000000, 49000000] # En mismo orden que Fx

long = False #True o False

NC = 99 #[0, 100]

VaR_CVaR_Fx(data, positions, NC, long)['Monetario ($)'][0]

46019.010965682115

# 7)

In [26]:
# Escribir listas en formato: 
#            ['S', 'K', 'r', 'sigma', 'T', 'N']

info_call = [[2694, 2650, 0.0439, 0.31, 3/12, 21],
             [2694, 2670, 0.0439, 0.28, 3/12, 25],
             [2694, 2650, 0.0421, 0.43, 12/12, 28],
             [2694, 2670, 0.0421, 0.37, 12/12, 33]]

info_put = [[2694, 2730, 0.0439, 0.35, 3/12, 45],
             [2694, 2740, 0.0439, 0.36, 3/12, 10],
             [2694, 2730, 0.0421, 0.51, 12/12, 12],
             [2694, 2740, 0.0421, 0.58, 12/12, 43]]

In [27]:
call_blsdelta = lambda S, k, r, sigma, T: norm.cdf((np.log(S/k) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T)))

put_blsdelta = lambda S, k, r, sigma, T: np.abs(norm.cdf((np.log(S/k) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))) - 1)

def delta_hedge(info_call, info_put):

    df_call = pd.DataFrame(info_call, columns=['S', 'K', 'r', 'sigma', 'T', 'N'])
    df_put = pd.DataFrame(info_put, columns=['S', 'K', 'r', 'sigma', 'T', 'N'])
    
    deltas_call_list = [call_blsdelta(*df_call.values[i][0:-1]) for i in range(len(df_call))]
    deltas_put_list = [put_blsdelta(*df_put.values[i][0:-1]) for i in range(len(df_call))]

    return np.dot(df_call.N, deltas_call_list) - np.dot(df_put.N, deltas_put_list)

In [32]:
net = delta_hedge(info_call, info_put)
print(f'Comprar {net:,.1f} millones de títulos')

Comprar 19.4 millones de títulos
